# Julia 機器學習：DecisionTree 決策樹

## 作業 030：乳癌預測資料集

請使用隨機森林模型建立一個分類模型，來預測乳癌資料集中，為良性或是惡性的腫瘤。

In [2]:
using Pkg
Pkg.add("DecisionTree")

   Updating registry at `C:\Users\chris-wang\.julia\registries\General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  99.9 %Fetching: [======>                                  ]  13.8 %=========>                               ]  20.3 %2 %===========>                             ]  26.5 %]  29.6 %]  32.2 % [===============>                         ]  35.4 % %2 %Fetching: [======================>                  ]  54.4 % [=========================>               ]  61.7 %>             ]  65.2 % %==============================>          ]  74.2 %.0 %=================================>       ]  81.6 %Fetching: [====================================>    ]  89.9 %93.5 % ]  96.8 %

  Resolving package versions...
  Installed Opus_jll ────── v1.3.1+1
  Installed libvorbis_jll ─ v1.3.6+3
  Installed OpenBLAS_jll ── v0.3.9+4
  Installed DecisionTree ── v0.10.1
   Updating `C:\Users\chris-wang\.julia\environments\v1.4\Project.toml`
  [7806a523] + DecisionTree v0.10.1
   Updating `C:\Users\chris-wang\.julia\environments\v1.4\Manifest.toml`
  [7806a523] + DecisionTree v0.10.1
  [4536629a] ↑ OpenBLAS_jll v0.3.9+2 ⇒ v0.3.9+4
  [91d4177d] ↑ Opus_jll v1.3.1+0 ⇒ v1.3.1+1
  [f27f6e37] ↑ libvorbis_jll v1.3.6+2 ⇒ v1.3.6+3


In [5]:
using ScikitLearn.CrossValidation: cross_val_score

┌ Info: Precompiling ScikitLearn [3646fa90-6ef7-5e7e-9f22-8aca16db6324]
└ @ Base loading.jl:1260


In [3]:
using DecisionTree, RDatasets, DataFrames, MLDataUtils, Statistics

┌ Info: Precompiling DecisionTree [7806a523-6efd-50cb-b5f6-3fa6f1930dbb]
└ @ Base loading.jl:1260


## 讀取資料

In [4]:
biopsy = dataset("MASS", "biopsy")

,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,Class
,String,Int32,Int32,Int32,Int32,Int32,Int32⍰,Int32,Int32,Int32,Categorical…
1,1000025,5,1,1,1,2,1,3,1,1,benign
2,1002945,5,4,4,5,7,10,3,2,1,benign
3,1015425,3,1,1,1,2,2,3,1,1,benign
4,1016277,6,8,8,1,3,4,3,7,1,benign
5,1017023,4,1,1,3,2,1,3,1,1,benign
6,1017122,8,10,10,8,7,10,9,7,1,malignant
7,1018099,1,1,1,1,2,10,3,1,1,benign
8,1018561,2,1,2,1,2,1,3,1,1,benign
9,1033078,2,1,1,1,2,1,1,1,5,benign


In [11]:
describe(biopsy,:nmissing)

,variable,nmissing
,Symbol,Union…
1,ID,
2,V1,
3,V2,
4,V3,
5,V4,
6,V5,
7,V6,16
8,V7,
9,V8,


┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[32]:1
└ @ Core In[32]:1


699-element Array{String,1}:
 "1000025"
 "1002945"
 "1015425"
 "1016277"
 "1017023"
 "1017122"
 "1018099"
 "1018561"
 "1033078"
 "1033078"
 "1035283"
 "1036172"
 "1041801"
 ⋮
 "566346"
 "603148"
 "654546"
 "654546"
 "695091"
 "714039"
 "763235"
 "776715"
 "841769"
 "888820"
 "897471"
 "897471"

In [33]:
for i in range(2,length(names(biopsy))-1)
    biopsy[i]=coalesce.(biopsy[i],mean(skipmissing(biopsy[i])))
end

┌ Warning: `range(start, stop)` (with neither `length` nor `step` given) is deprecated, use `range(start, stop=stop)` instead.
│   caller = top-level scope at In[33]:1
└ @ Core .\In[33]:1
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[33]:2
└ @ Core .\In[33]:2
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[33]:2
└ @ Core .\In[33]:2
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at In[33]:2
└ @ Core .\In[33]:2


In [35]:
describe(biopsy,:nmissing)

,variable,nmissing
,Symbol,Nothing
1,ID,
2,V1,
3,V2,
4,V3,
5,V4,
6,V5,
7,V6,
8,V7,
9,V8,


In [14]:
biopsy_2=dropmissing(biopsy, :V6);

In [15]:
features= Matrix(biopsy_2[!,2:10])
labels= string.(biopsy_2[:Class])

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[15]:2
└ @ Core In[15]:2


683-element Array{String,1}:
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"
 "malignant"
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"
 "malignant"
 ⋮
 "benign"
 "benign"
 "benign"
 "benign"
 "malignant"
 "benign"
 "benign"
 "benign"
 "benign"
 "malignant"
 "malignant"
 "malignant"

In [36]:
#以平均值填補缺值
features= Matrix(biopsy[!,2:10])
labels= string.(biopsy[:Class])

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[36]:3
└ @ Core In[36]:3


699-element Array{String,1}:
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"
 "malignant"
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"
 "malignant"
 ⋮
 "benign"
 "benign"
 "benign"
 "benign"
 "malignant"
 "benign"
 "benign"
 "benign"
 "benign"
 "malignant"
 "malignant"
 "malignant"

In [37]:
model= DecisionTree.DecisionTreeClassifier(max_depth=2)

DecisionTreeClassifier
max_depth:                2
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  nothing
root:                     nothing

In [38]:
DecisionTree.fit!(model, features, labels)

DecisionTreeClassifier
max_depth:                2
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  ["benign", "malignant"]
root:                     Decision Tree
Leaves: 4
Depth:  2

In [39]:
DecisionTree.get_classes(model)

2-element Array{String,1}:
 "benign"
 "malignant"

In [40]:
accuracy = cross_val_score(model, features, labels, cv=5)

5-element Array{Float64,1}:
 0.9148936170212766
 0.9142857142857143
 0.8785714285714286
 0.9424460431654677
 0.8920863309352518